<a href="https://colab.research.google.com/github/heyimknny/DS2002/blob/main/Finance_API_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Finance API Tutorial - Homework Assignment - Kenneth Nguyen
## Objective

In this assignment, you will:
1. Request stock data for two or more stock symbols.
2. Extract and display only the Stock Ticker, Company Name, and Current Market Price.
3. Allow the user to call another endpoint with a module and store the response in a
DataFrame.

In [ ]:
import requests
import pandas as pd

# Manually define API key [will delete on final submission]
API_KEY = ""

## Task 1: Fetch Basic Stock Data
- Prompt the user to input two or more stock symbols.
- Use the API to fetch data for the given symbols.
- Extract and display only the Stock Ticker, Company Name, and Current Market Price.

In [ ]:
BASE_URL = "https://yfapi.net/v6/finance/quote"

In [ ]:
# Use the API to fetch data for the given symbols.
def get_stock_data(symbols):
    querystring = {"symbols": ",".join(symbols)}
    headers = {'x-api-key': API_KEY}
    response = requests.get(BASE_URL, headers=headers, params=querystring)

    if response.status_code == 200:
        data = response.json()["quoteResponse"]["result"]
        stock_info = [{
            "Stock Ticker": item["symbol"],
            "Company": item.get("shortName", "N/A"),
            "Current Market Price": f"${item.get('regularMarketPrice', 'N/A')}"
        } for item in data]

        # Extract and display only the Stock Ticker, Company Name, and Current Market Price.
        for stock in stock_info:
            print(f"Stock Ticker: {stock['Stock Ticker']}, Company: {stock['Company']}, Current Market Price: {stock['Current Market Price']}")

    else:
        print("Failed to fetch stock data")

Example Input:

AAPL, MSFT

Example Output:

Stock Ticker: AAPL, Company: Apple Inc., Current Market Price: $175.23

Stock Ticker: MSFT, Company: Microsoft Corp., Current Market Price: $310.45

In [ ]:
# Prompt the user to input two or more stock symbols.
# NOTE: Requires a comma AND a space between stock symbols.
stock_symbols = input("Enter stock symbols separated by commas: ").upper().split(', ')
get_stock_data(stock_symbols)

Enter stock symbols separated by commas: AAPL, MSFT
Stock Ticker: AAPL, Company: Apple Inc., Current Market Price: $247.04
Stock Ticker: MSFT, Company: Microsoft Corporation, Current Market Price: $397.9


## Task 2.1: Fetch Additional Data Using Modules
- Allow the user to choose a module from the Quote Summary Endpoint.
Find:
52 Week High
52 Week Low
Return on Assets (ROA)
- Make an API request using the chosen module.
- Convert the response into a Pandas DataFrame.
- Display the DataFrame.

In [ ]:
BASE_URL = "https://yfapi.net/v11/finance/quoteSummary"

In [ ]:
# Allow the user to choose a module from the Quote Summary Endpoint.
def get_additional_data(symbols, metric):
    module_map = {
        "52 Week High": ("summaryDetail", "fiftyTwoWeekHigh"),
        "52 Week Low": ("summaryDetail", "fiftyTwoWeekLow"),
        "Return on Assets": ("financialData", "returnOnAssets")
    }

    if metric not in module_map:
        print("Invalid metric selected.")
        return

    module_name, metric_key = module_map[metric]
    stock_data = []

    for symbol in symbols:
        querystring = {"modules": module_name}
        headers = {'x-api-key': API_KEY}

        response = requests.get(f"{BASE_URL}/{symbol}", headers=headers, params=querystring)

        if response.status_code == 200:
            data = response.json().get("quoteSummary", {}).get("result", [])

            if not data or module_name not in data[0]:
                print(f"No data found for {symbol}.")
                continue

            stock_info = {
                "Stock Ticker": symbol,
                metric: data[0][module_name].get(metric_key, {}).get("raw", "N/A")
            }
            stock_data.append(stock_info)

        else:
            print(f"Failed to fetch data for {symbol}. Status Code: {response.status_code}")

    if stock_data:
        df = pd.DataFrame(stock_data)
        print(df)
    else:
        print("No valid data to display.")

In [ ]:
# Prompt the user to input two or more stock symbols and then choose a module
symbols = input("Enter stock symbols separated by commas: ").upper().split(',')
metric = input("Choose a metric (52 Week High, 52 Week Low, Return on Assets): ")
get_additional_data(symbols, metric)

Enter stock symbols separated by commas: AAPL
Choose a metric (52 Week High, 52 Week Low, Return on Assets): 52 Week High
  Stock Ticker  52 Week High
0         AAPL         260.1


## Task 2.2 Fetch Additional Data Using Modules
Find Current Trending Stocks , Their Actual Name and Ticker and Current Price as well as their 52 high and low.

In [ ]:
TRENDING_URL = "https://yfapi.net/v1/finance/trending/US"
QUOTE_SUMMARY_URL = "https://yfapi.net/v11/finance/quoteSummary"

In [ ]:
def get_trending_stocks():
    headers = {'x-api-key': API_KEY}
    response = requests.get(TRENDING_URL, headers=headers)

    if response.status_code == 200:
        data = response.json().get("finance", {}).get("result", [])

        if not data:
            print("No trending stocks found.")
            return []

        # grab stock symbols from trending list
        return [item["symbol"] for item in data[0]["quotes"]]

    else:
        print("Failed to fetch trending stocks.")
        return []

def get_stock_details(symbols):
    stock_data = []

    for symbol in symbols:
        querystring = {"modules": "price,summaryDetail"}
        headers = {'x-api-key': API_KEY}
        response = requests.get(f"{QUOTE_SUMMARY_URL}/{symbol}", headers=headers, params=querystring)

        if response.status_code == 200:
            data = response.json().get("quoteSummary", {}).get("result", [])

            if not data:
                print(f"No data found for {symbol}.")
                continue

            # get details of stocks
            stock_info = {
                "Stock Ticker": symbol,
                "Company": data[0]["price"].get("longName", "N/A"),
                "Current Price": f"${data[0]['price'].get('regularMarketPrice', {}).get('raw', 'N/A')}",
                "52 Week High": data[0]["summaryDetail"].get("fiftyTwoWeekHigh", {}).get("raw", "N/A"),
                "52 Week Low": data[0]["summaryDetail"].get("fiftyTwoWeekLow", {}).get("raw", "N/A")
            }
            stock_data.append(stock_info)

        else:
            print(f"Failed to fetch details for {symbol}. Status Code: {response.status_code}")

    if stock_data:
        df = pd.DataFrame(stock_data)
        print(df)
    else:
        print("No valid data to display.")

In [ ]:
trending_symbols = get_trending_stocks()

if trending_symbols:
    get_stock_details(trending_symbols)
else:
    print("No trending stocks to display.")

   Stock Ticker                                 Company Current Price  \
0          SMCI              Super Micro Computer, Inc.        $45.54   
1          TSLA                             Tesla, Inc.        $302.8   
2          LCID                       Lucid Group, Inc.         $2.61   
3          CAVA                        CAVA Group, Inc.         $99.3   
4          WDAY                           Workday, Inc.       $255.22   
5          NVDA                      NVIDIA Corporation       $126.63   
6          ONVO                 Organovo Holdings, Inc.         $1.27   
7          LMND                          Lemonade, Inc.        $31.99   
8          FSLR                       First Solar, Inc.       $147.46   
9          AXON                   Axon Enterprise, Inc.       $496.65   
10         INTU                             Intuit Inc.       $555.63   
11          AMC        AMC Entertainment Holdings, Inc.         $3.27   
12         CPNG                           Coupang, 

## Bonus Challenge
- Allow users to fetch data for different modules dynamically.
- Format currency values properly (e.g., `$1,500,000,000` instead of `1500000000`).


In [ ]:
MODULE_MAP = {
    "52 Week High": "summaryDetail",
    "52 Week Low": "summaryDetail",
    "Return on Assets": "financialData"
}

# format as currency if number, else N/A
def format_currency(value):
    if isinstance(value, (int, float)):
        return f"${value:,.2f}"
    return "N/A"

def get_stock_details(symbols, modules):
    stock_data = []

    # map user-selected modules to API modules
    selected_modules = [MODULE_MAP.get(module, "") for module in modules if module in MODULE_MAP]

    if not selected_modules:
        print("No valid modules selected.")
        return pd.DataFrame()

    # add price module to always get company name + price
    querystring = {"modules": ",".join(set(selected_modules + ["price"]))}
    headers = {'x-api-key': API_KEY}

    for symbol in symbols:
        response = requests.get(f"{QUOTE_SUMMARY_URL}/{symbol}", headers=headers, params=querystring)

        if response.status_code == 200:
            data = response.json().get("quoteSummary", {}).get("result", [])

            if not data:
                print(f"No data found for {symbol}.")
                continue

            item = data[0]
            price_info = item.get("price", {})
            company_name = price_info.get("longName", "N/A")
            current_price = format_currency(price_info.get("regularMarketPrice", {}).get("raw", "N/A"))

            stock_info = {
                "Stock Ticker": symbol,
                "Company": company_name,
                "Current Price": current_price,
            }

            # pull requested modules from API response
            for module in modules:
                if module == "52 Week High":
                    stock_info["52 Week High"] = format_currency(
                        item.get("summaryDetail", {}).get("fiftyTwoWeekHigh", {}).get("raw", "N/A")
                    )
                elif module == "52 Week Low":
                    stock_info["52 Week Low"] = format_currency(
                        item.get("summaryDetail", {}).get("fiftyTwoWeekLow", {}).get("raw", "N/A")
                    )
                elif module == "Return on Assets":
                    return_on_assets = item.get("financialData", {}).get("returnOnAssets", {}).get("raw")
                    stock_info["Return on Assets"] = f"{return_on_assets * 100:.2f}%" if return_on_assets else "N/A"

            stock_data.append(stock_info)

        else:
            print(f"Failed to fetch details for {symbol}. Status Code: {response.status_code}")

    return pd.DataFrame(stock_data)

trending_symbols = get_trending_stocks()

# user picks what modules
if trending_symbols:
    print("Available Modules: 52 Week High, 52 Week Low, Return on Assets")
    selected_modules = input("Enter the modules you want (comma-separated): ").split(", ")

    stock_df = get_stock_details(trending_symbols, selected_modules)
    print(stock_df)
else:
    print("No trending stocks to display.")

Available Modules: 52 Week High, 52 Week Low, Return on Assets
Enter the modules you want (comma-separated): 52 Week High, 52 Week Low
   Stock Ticker                                 Company Current Price  \
0          SMCI              Super Micro Computer, Inc.        $45.54   
1          TSLA                             Tesla, Inc.       $302.80   
2          LCID                       Lucid Group, Inc.         $2.61   
3          CAVA                        CAVA Group, Inc.        $99.30   
4          WDAY                           Workday, Inc.       $255.22   
5          NVDA                      NVIDIA Corporation       $126.63   
6          ONVO                 Organovo Holdings, Inc.         $1.27   
7          LMND                          Lemonade, Inc.        $31.99   
8          FSLR                       First Solar, Inc.       $147.46   
9          AXON                   Axon Enterprise, Inc.       $496.65   
10         INTU                             Intuit Inc.       

# Bonus Challenge 2: Where is Elon’s Plane Right NOW?

In [4]:
import requests
import datetime

URL = "https://opensky-network.org/api/states/all"

ICAO24 = "a835af"

def get_musk_plane_location():
    response = requests.get(URL)

    if response.status_code == 200:
        data = response.json()

        flights = data.get("states", [])

        for flight in flights:
            if flight[0].lower() == ICAO24:
                timestamp = datetime.datetime.utcfromtimestamp(flight[3]).strftime('%Y-%m-%d %H:%M:%S UTC')
                latitude = flight[6]
                longitude = flight[5]
                altitude = flight[7]
                velocity = flight[9]

                return {
                    "Timestamp": timestamp,
                    "Latitude": latitude,
                    "Longitude": longitude,
                    "Altitude (m)": altitude,
                    "Velocity (m/s)": velocity
                }

        return "Elon Musk's plane is not currently tracked."

    else:
        return f"Failed to fetch data, HTTP Status Code: {response.status_code}"

plane_data = get_musk_plane_location()
print(plane_data)

Elon Musk's plane is not currently tracked.


According to https://opensky-network.org/aircraft-profile?icao24=a835af, the plane is not being tracked.